## Plan
### Turning tweets into features

- Start with trigrams, can tune later
- Can consider bigrams, bag of words, or other n-grams
- Ignore location information, at least for now
- Almost all tweets have keywords, use as another feature
- Make sure to process "keyword" values, removing special characters

### Criteria for disaster
- Meant to track if tweets are referring to ongoing disasters
- Also includes historical events


### Training
- Train and validate our model on `train.csv` 
- Test by sending results to Kaggle

### Random forest
- Use Gini criterion for efficiency

## Importing and vectorizing data

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import string
import numpy as np
import pandas as pd
import re

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
count_vectorizer = CountVectorizer()

In [4]:
def standardize_string(s):
    s = s.lower()
    s = re.sub("http://t\.co/\S+", "", s)
    return s

In [5]:
print(standardize_string("http://t.co/"))

http://t.co/


In [6]:
all_characters = set()

for tweet in train_df['text']:
    all_characters = all_characters.union(set(standardize_string(tweet)))

char_list = list(all_characters)
char_list.sort()
print(char_list)

['\n', ' ', '!', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x89', '\x9d', '¡', '¢', '£', '¤', '¨', '©', 'ª', '«', '¬', '´', '¼', 'â', 'ã', 'å', 'ç', 'è', 'ê', 'ì', 'ï', 'ñ', 'ò', 'ó', '÷', 'û', 'ü']


In [7]:
included_chars = list(string.ascii_lowercase + string.digits) + ['#', '@', 'â', 'ã', 'å', 'ç', 'è', 'ê', 'ì', 'ï', 'ñ', 'ò', 'ó', 'ü', ' ']
print(included_chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '#', '@', 'â', 'ã', 'å', 'ç', 'è', 'ê', 'ì', 'ï', 'ñ', 'ò', 'ó', 'ü', ' ']


### Stripping characters
- Try both with and without removing special characters
- Consider skipping data points with bad characters

In [8]:
def remove_special_characters(s):
    for c in char_list:
        if c not in included_chars:
            s = s.replace(c, "")
    return s

In [9]:
def tweet_to_array(t):
    t = standardize_string(t)
    tweet_array = t.split()
    return tweet_array

In [10]:
def tokenize_data():
    all_tweets = []
    for tweet in train_df['text']:
        tweet_array = tweet_to_array(tweet)
        invalid_fields = []
        for field in tweet_array:
            for c in excluded_chars:
                if c in field:
                    invalid_fields.append(field)
                    continue
        
        for f in invalid_fields:
            del f
        all_tweets.append(tweet_array)
        
    return all_tweets   

In [29]:
# From Jarem:
# mock-up of random forest, using these sites as reference:
# https://machinelearningmastery.com/random-forest-ensemble-in-python/
# https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook
# getting super bad performance - worse than chance often - and I don't know much hyperparameter tuning will fix it
# not sure what I'm doing wrong here - n-grams and text formatting seem to make it worse

def format_tweet(t):
    # Makes lowercase
    formatted_tweet = t.lower()
    # Removed links
    formatted_tweet = re.sub(" http(s|)://t\.co/\S+", "", formatted_tweet)
    formatted_tweet = re.sub("http(s|)://t\.co/\S+", "", formatted_tweet)
    # Removes any special characters, other than a-z, numbers, spaces, hashtags, and @
    formatted_tweet = remove_special_characters(formatted_tweet)
    final_tweet_array = []
    
    # Removes multiple consecutive spaces
    for i, char in enumerate(formatted_tweet):
        if i == 0:
            if char != ' ':
                final_tweet_array.append(char)
                continue
        prev_char = formatted_tweet[i-1]
        if char == ' ' and prev_char == ' ':
            continue
        final_tweet_array.append(char)
    final_tweet = "".join(final_tweet_array)
    return final_tweet

formatted_train_tweets = []

#basic string formatting
for i, tweet in enumerate(train_df["text"]):
    formatted_train_tweets.append(format_tweet(tweet))

count_vectorizer = CountVectorizer(ngram_range=(2,2))
train_vectors = count_vectorizer.fit_transform(formatted_train_tweets)

formatted_test_tweets = []

for tweet in test_df["text"]:
    formatted_test_tweets.append(format_tweet(tweet))
    
test_ids = test_df['id']

test_vectors = count_vectorizer.transform(formatted_test_tweets)


# 1-gram no string formatting
# array([0.55543823, 0.50891089, 0.54221388, 0.51913133, 0.68794326])
# 1 and 2-gram, no string formatting
# array([0.46118721, 0.45027322, 0.43412527, 0.44141069, 0.61523626])
# 1-gram basic string formatting
# array([0.57556936, 0.48219736, 0.5530303 , 0.51859504, 0.68586387])
# 1 & 2-gram, basic string formatting
# array([0.5039019 , 0.41150442, 0.41241685, 0.45823928, 0.62327416])
# bigram only, basic string formatting
# array([0.24096386, 0.25725095, 0.1682243 , 0.17475728, 0.31060606])

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier(n_jobs=10, max_depth=None, class_weight="balanced")

rf_parameters = {
    'min_samples_split': range(2, 5),
    'min_samples_leaf': range(1, 4),
    'n_estimators': [50, 100, 500]
}


rf_cv = GridSearchCV(rf, parameters, verbose=3, n_jobs=10)
rf_cv.fit(train_vectors, train_df['target'])
print(rf_cv.best_params_)

# {'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 500}
# {'class_weight': 'balanced', 'max_depth': None, 'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1}

Fitting 5 folds for each of 27 candidates, totalling 135 fits


KeyboardInterrupt: 

In [20]:
rf_predicted_classes = rf_cv.predict(test_vectors)
print(rf_predicted_classes)
rf_out_array = []
for i, pred_class in enumerate(rf_predicted_classes):
    rf_out_array.append([int(test_ids[i]), pred_class])

np.savetxt("rf-results.csv", rf_out_array, delimiter=',', fmt='%i')

[0 0 1 ... 1 1 1]
[CV 1/5] END min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.641 total time= 1.3min
[CV 5/5] END min_samples_leaf=1, min_samples_split=3, n_estimators=50;, score=0.667 total time=  35.2s
[CV 5/5] END min_samples_leaf=1, min_samples_split=3, n_estimators=100;, score=0.654 total time= 1.2min
[CV 4/5] END min_samples_leaf=1, min_samples_split=3, n_estimators=500;, score=0.611 total time= 5.9min
[CV 5/5] END min_samples_leaf=2, min_samples_split=2, n_estimators=50;, score=0.671 total time=  24.3s
[CV 5/5] END min_samples_leaf=2, min_samples_split=2, n_estimators=100;, score=0.671 total time=  46.4s
[CV 5/5] END min_samples_leaf=2, min_samples_split=2, n_estimators=500;, score=0.670 total time= 3.8min
[CV 2/5] END min_samples_leaf=2, min_samples_split=3, n_estimators=100;, score=0.630 total time=  42.6s
[CV 1/5] END min_samples_leaf=2, min_samples_split=4, n_estimators=50;, score=0.649 total time=  22.7s
[CV 1/5] END min_samples_leaf=2, min_samples_spli

[CV 2/5] END min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.610 total time=  36.3s
[CV 2/5] END min_samples_leaf=1, min_samples_split=2, n_estimators=500;, score=0.618 total time= 5.9min
[CV 2/5] END min_samples_leaf=1, min_samples_split=4, n_estimators=50;, score=0.621 total time=  32.6s
[CV 1/5] END min_samples_leaf=1, min_samples_split=4, n_estimators=100;, score=0.640 total time= 1.2min
[CV 2/5] END min_samples_leaf=1, min_samples_split=4, n_estimators=500;, score=0.619 total time= 5.6min
[CV 5/5] END min_samples_leaf=2, min_samples_split=3, n_estimators=50;, score=0.658 total time=  21.5s
[CV 3/5] END min_samples_leaf=2, min_samples_split=3, n_estimators=500;, score=0.622 total time= 3.6min
[CV 2/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=50;, score=0.624 total time=  10.2s
[CV 5/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=50;, score=0.665 total time=  11.8s
[CV 5/5] END min_samples_leaf=3, min_samples_split=2, n_estimators=10

In [35]:
from sklearn.neural_network import MLPClassifier

mlpc = MLPClassifier(verbose=True, tol=.001)
mlpc_parameters = {
    "alpha": [.0001, .001, .01, .1]
}
mlpc_cv = GridSearchCV(mlpc, mlpc_parameters, verbose=3, n_jobs=-1)
mlpc_cv.fit(train_vectors, train_df['target'])
print(mlpc_cv.best_params_)

mlpc_predicted_classes = mlpc_cv.predict(test_vectors)
mlpc_out_array = []
for i, pred_class in enumerate(mlpc_predicted_classes):
    mlpc_out_array.append([int(test_ids[i]), pred_class])
    
np.savetxt("mlpc-results.csv", mlpc_out_array, delimiter=',', fmt='%i')

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Iteration 1, loss = 0.64550188
Iteration 2, loss = 0.37110657
Iteration 3, loss = 0.16113751
Iteration 4, loss = 0.09245533
Iteration 5, loss = 0.06919119
Iteration 6, loss = 0.05685333
Iteration 7, loss = 0.05051970
Iteration 8, loss = 0.04652030
Iteration 9, loss = 0.04329465
Iteration 10, loss = 0.04027972
Iteration 11, loss = 0.03963507
Iteration 12, loss = 0.03706136
Iteration 13, loss = 0.03634189
Iteration 14, loss = 0.03578973
Iteration 15, loss = 0.03438279
Iteration 16, loss = 0.03405455
Iteration 17, loss = 0.03368897
Iteration 18, loss = 0.03330628
Iteration 19, loss = 0.03267945
Iteration 20, loss = 0.03235284
Iteration 21, loss = 0.03212069
Iteration 22, loss = 0.03165786
Iteration 23, loss = 0.03103042
Iteration 24, loss = 0.03137677
Iteration 25, loss = 0.03200844
Iteration 26, loss = 0.03082211
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
{'alpha': 0.0001}
